In [1]:
!curl https://certs.secureserver.net/repository/sf-class2-root.crt -O

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1468  100  1468    0     0   4743      0 --:--:-- --:--:-- --:--:--  4750


In [2]:
%pip install cassandra-sigv4

  Using cached cassandra_sigv4-4.0.2-py2.py3-none-any.whl.metadata (5.9 kB)
  Using cached cassandra_driver-3.29.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.2 kB)
  Using cached boto3-1.38.12-py3-none-any.whl.metadata (6.6 kB)
  Using cached botocore-1.38.12-py3-none-any.whl.metadata (5.7 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.12.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached geomet-0.2.1.post1-py3-none-any.whl.metadata (1.0 kB)
Using cached cassandra_sigv4-4.0.2-py2.py3-none-any.whl (9.8 kB)
Using cached boto3-1.38.12-py3-none-any.whl (139 kB)
Using cached cassandra_driver-3.29.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.9 MB)
Using cached botocore-1.38.12-py3-none-any.whl (13.5 MB)
Using cached geomet-0.2.1.post1-py3-none-any.whl (18 kB)
Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
Using cached s3transfer-0.12.0-py3-none-any.whl (84 kB)
Note: you may need to restart

In [3]:
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1_2, CERT_REQUIRED
from cassandra_sigv4.auth import SigV4AuthProvider
import boto3

# ssl setup
ssl_context = SSLContext(PROTOCOL_TLSv1_2)
ssl_context.load_verify_locations('sf-class2-root.crt')  # change your file path for locating the certificate
ssl_context.verify_mode = CERT_REQUIRED

# boto3 session setup
boto_session = boto3.Session(region_name="us-east-2")  # this AWS credentials is specific to `us-east-2` region

/tmp/ipykernel_124/332746716.py:7: DeprecationWarning: ssl.PROTOCOL_TLSv1_2 is deprecated
  ssl_context = SSLContext(PROTOCOL_TLSv1_2)


In [4]:
# authorization setup with SigV4
auth_provider = SigV4AuthProvider(boto_session)

In [5]:
#cluster setup 
cluster = Cluster(['cassandra.us-east-2.amazonaws.com'], 
                  ssl_context=ssl_context, 
                  auth_provider=auth_provider, 
                  port=9142)  # TLS only communicates on port 9142

In [6]:
# establishing connection to Keyspace
session = cluster.connect()

In [7]:
# For example, create a keyspace for HW2
r = session.execute('''
    CREATE KEYSPACE IF NOT EXISTS de300_hw2 
    WITH replication = {'class': 'SingleRegionStrategy'};
    ''')
print(r.current_rows)

[]


In [8]:
from cassandra.cluster import ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra import ConsistencyLevel


# Define execution profile with LOCAL_QUORUM
execution_profile = ExecutionProfile(
    consistency_level=ConsistencyLevel.LOCAL_QUORUM
)

# Cluster setup with correct profile
cluster = Cluster(
    ['cassandra.us-east-2.amazonaws.com'],
    ssl_context=ssl_context,
    auth_provider=auth_provider,
    port=9142,
    execution_profiles={EXEC_PROFILE_DEFAULT: execution_profile}
)

# establishing connection to Keyspace
session = cluster.connect()
session.set_keyspace('de300_hw2')  # Replace with your keyspace

In [9]:
import pandas as pd

# This is the path to the CSV file inside the container
admissions_w_age = pd.read_csv("ADMISSIONS_W_AGE.csv")
d_icd_proc = pd.read_csv("D_ICD_PROCEDURES.csv")
drgcodes = pd.read_csv("DRGCODES.csv")
icustays = pd.read_csv("ICUSTAYS.csv")
patients = pd.read_csv("PATIENTS.csv")
prescriptions = pd.read_csv("PRESCRIPTIONS.csv")
procs_icd = pd.read_csv("PROCEDURES_ICD.csv")
admissions = pd.read_csv("ADMISSIONS.csv")
icustays_hours = pd.read_csv("ICUSTAYS_W_HOURS.csv")

Question 1:

In [25]:
#!pip install duckdb
import duckdb
import pandas as pd

con = duckdb.connect()
con.register('prescriptions', prescriptions)
con.register('admissions_w_age', admissions_w_age)


ethnicity_drug = con.execute(
    """
        SELECT *
        FROM PRESCRIPTIONS
        JOIN ADMISSIONS_W_AGE
        ON PRESCRIPTIONS.hadm_id = ADMISSIONS_W_AGE.hadm_id
   """ ).df()

ethnicity_drug.head()

,row_id,subject_id,hadm_id,icustay_id,startdate,enddate,drug_type,drug,drug_name_poe,drug_name_generic,...,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data,age_group
0,32600,42458,159647,NaN,2146-07-21 00:00:00,2146-07-22 00:00:00,MAIN,Pneumococcal Vac Polyvalent,Pneumococcal Vac Polyvalent,PNEUMOcoccal Vac Polyvalent,...,ENGL,JEWISH,MARRIED,WHITE,2146-07-21 11:13:00,2146-07-21 15:59:00,PNEUMONIA,0,1,80 or older
1,32601,42458,159647,NaN,2146-07-21 00:00:00,2146-07-22 00:00:00,MAIN,Bisacodyl,Bisacodyl,Bisacodyl,...,ENGL,JEWISH,MARRIED,WHITE,2146-07-21 11:13:00,2146-07-21 15:59:00,PNEUMONIA,0,1,80 or older
2,32602,42458,159647,NaN,2146-07-21 00:00:00,2146-07-22 00:00:00,MAIN,Bisacodyl,Bisacodyl,Bisacodyl (Rectal),...,ENGL,JEWISH,MARRIED,WHITE,2146-07-21 11:13:00,2146-07-21 15:59:00,PNEUMONIA,0,1,80 or older
3,32603,42458,159647,NaN,2146-07-21 00:00:00,2146-07-22 00:00:00,MAIN,Senna,Senna,Senna,...,ENGL,JEWISH,MARRIED,WHITE,2146-07-21 11:13:00,2146-07-21 15:59:00,PNEUMONIA,0,1,80 or older
4,32604,42458,159647,NaN,2146-07-21 00:00:00,2146-07-21 00:00:00,MAIN,Docusate Sodium (Liquid),Docusate Sodium (Liquid),Docusate Sodium (Liquid),...,ENGL,JEWISH,MARRIED,WHITE,2146-07-21 11:13:00,2146-07-21 15:59:00,PNEUMONIA,0,1,80 or older


In [ ]:
session.execute("""
    CREATE TABLE IF NOT EXISTS de300_hw2.ethnicity_drgtype (
        unique_id uuid,             
        ethnicity text,
        drug_type text,
        PRIMARY KEY ((ethnicity), unique_id)
    ) WITH CLUSTERING ORDER BY (unique_id DESC);
    """)

In [ ]:
import uuid

for _, row in ethnicity_drug.iterrows():
    session.execute("""
        INSERT INTO de300_hw2.ethnicity_drgtype (unique_id, ethnicity, drug_type)
        VALUES (%s, %s, %s)
    """, (uuid.uuid4(), row['ethnicity'], row['drug_type']))


In [ ]:
from cassandra.query import SimpleStatement

select = SimpleStatement(
    "SELECT ethnicity, drug_type FROM de300_hw2.ethnicity_drgtype",
    consistency_level=ConsistencyLevel.LOCAL_QUORUM
)
rows = session.execute(select)
ethnicity_drug_df = pd.DataFrame(rows)

In [91]:
counts_by_ethnicity = (
    ethnicity_drug_df
    .groupby(['ethnicity', 'drug_type'])
    .size()
    .reset_index(name='total_amount')
    .sort_values(['total_amount'], ascending= False)
)

print("Most Used Drug Type Use by Ethnicity:")
print(counts_by_ethnicity.groupby('ethnicity').head(1))


Most Used Drug Type Use by Ethnicity:
                                            ethnicity drug_type  total_amount
20                                              WHITE      MAIN          5420
10                     HISPANIC/LATINO - PUERTO RICAN      MAIN           860
6                              BLACK/AFRICAN AMERICAN      MAIN           476
17                              UNKNOWN/NOT SPECIFIED      MAIN           245
8                                  HISPANIC OR LATINO      MAIN           226
2   AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...      MAIN           200
4                                               ASIAN      MAIN           121
15                                   UNABLE TO OBTAIN      MAIN            89
12                                              OTHER      MAIN            72


Question 2:

In [10]:
!pip install duckdb
import duckdb
import pandas as pd

con = duckdb.connect()
con.register('procs_icd', procs_icd)
con.register('d_icd_proc', d_icd_proc)
con.register('admissions_w_age', admissions_w_age)

proc_with_age = con.execute("""
    SELECT 
        procs_icd.*, 
        d_icd_proc.short_title,
        admissions_w_age.age_group,
    FROM procs_icd 
    JOIN d_icd_proc ON procs_icd.icd9_code = d_icd_proc.icd9_code
    JOIN admissions_w_age ON procs_icd.hadm_id = admissions_w_age.hadm_id
""").df()


  Using cached duckdb-1.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (966 bytes)
Using cached duckdb-1.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (20.2 MB)


In [15]:
session.execute("""
    CREATE TABLE IF NOT EXISTS de300_hw2.agegroup_proc (
        unique_id uuid,             
        age_group text,
        subject_id int,
        short_title text,
        icd9_code int,
        PRIMARY KEY ((age_group), unique_id)
    ) WITH CLUSTERING ORDER BY (unique_id DESC);
""")


In [16]:
import uuid

for _, row in proc_with_age.iterrows():
    session.execute("""
        INSERT INTO de300_hw2.agegroup_proc (unique_id, age_group, subject_id, short_title, icd9_code)
        VALUES (%s, %s, %s, %s, %s)
    """, (uuid.uuid4(), row['age_group'], row['subject_id'], row['short_title'], row['icd9_code']))


In [11]:
from cassandra.query import SimpleStatement

select = SimpleStatement(
    "SELECT age_group, short_title, icd9_code as procedure FROM de300_hw2.agegroup_proc",
    consistency_level=ConsistencyLevel.LOCAL_QUORUM
)
rows = session.execute(select)
age_drug_df = pd.DataFrame(rows)

In [24]:
counts_by_agegroup = (
    age_drug_df
    .groupby(['age_group', 'short_title'])
    .size()
    .reset_index(name='procedure_count')
    .sort_values(['age_group', 'procedure_count'], ascending=[True, False])
)

print("Most frequent procedures by age group:")
print(counts_by_agegroup.groupby('age_group').head(3))


Most frequent procedures by age group:
         age_group               short_title  procedure_count
17   19 or younger           Venous cath NEC                2
0    19 or younger  Applic ext fix dev-femur                1
1    19 or younger         Atlas-axis fusion                1
66           20-49           Venous cath NEC                9
32           20-49   Entral infus nutrit sub                7
28           20-49  Cont inv mec ven 96+ hrs                6
165          50-79           Venous cath NEC               25
98           50-79   Entral infus nutrit sub               22
129          50-79   Packed cell transfusion               13
235    80 or older           Venous cath NEC               20
214    80 or older   Packed cell transfusion               13
195    80 or older  Insert endotracheal tube                8


Question 3:

In [90]:
#!pip install duckdb
import duckdb
import pandas as pd

con = duckdb.connect()
con.register('icustays_hours', icustays_hours)
con.register('patients', patients)
con.register('admissions', admissions)


icuhours_gend_eth = con.execute(
    """
        SELECT *
        FROM ICUSTAYS_HOURS JOIN PATIENTS
        ON ICUSTAYS_HOURS.subject_id = PATIENTS.subject_id
        JOIN ADMISSIONS ON ICUSTAYS_HOURS.hadm_id = ADMISSIONS.hadm_id
   """ ).df()


In [52]:
session.execute("""DROP TABLE de300_hw2.icustayhrs""")

In [57]:
session.execute("""
    CREATE TABLE IF NOT EXISTS de300_hw2.icustayhrs (
        unique_id uuid,             
        ethnicity text,
        gender text,
        hours_of_stay float,
        PRIMARY KEY ((gender), ethnicity, unique_id)
    ) WITH CLUSTERING ORDER BY (ethnicity DESC);
""")

In [58]:
import uuid

for _, row in icuhours_gend_eth.iterrows():
    session.execute("""
        INSERT INTO de300_hw2.icustayhrs (unique_id, ethnicity, gender, hours_of_stay)
        VALUES (%s, %s, %s, %s)
    """, (uuid.uuid4(), row['ethnicity'], row['gender'], row['hours_of_stay']))


In [76]:
from cassandra.query import SimpleStatement

select = SimpleStatement(
    "SELECT ethnicity, gender, hours_of_stay FROM de300_hw2.icustayhrs",
    consistency_level=ConsistencyLevel.LOCAL_QUORUM
)
rows = session.execute(select)
ethnicity_gend_stay = pd.DataFrame(rows)

In [89]:
stay_by_eth_and_gend = (
    ethnicity_gend_stay
    .dropna(subset=['hours_of_stay']) 
    .groupby(['ethnicity', 'gender'])['hours_of_stay']
    .mean()
    .round(1)
    .reset_index()
    .sort_values(by=['ethnicity', 'hours_of_stay'], ascending=[False, False])
)

print("Average hours of ICU stay by gender and ethnicity:")
print(stay_by_eth_and_gend)

Average hours of ICU stay by gender and ethnicity:
                                            ethnicity gender  hours_of_stay
12                                              WHITE      F          124.7
13                                              WHITE      M           75.7
10                              UNKNOWN/NOT SPECIFIED      F          133.0
11                              UNKNOWN/NOT SPECIFIED      M           51.5
9                                    UNABLE TO OBTAIN      M          320.6
7                                               OTHER      F           32.1
8                                               OTHER      M            2.5
6                      HISPANIC/LATINO - PUERTO RICAN      M           77.8
5                                  HISPANIC OR LATINO      F          179.0
3                              BLACK/AFRICAN AMERICAN      F          268.9
4                              BLACK/AFRICAN AMERICAN      M           71.4
2                                    

In [87]:
stay_by_eth = (
    ethnicity_gend_stay 
    .dropna(subset=['hours_of_stay']) 
    .groupby(['ethnicity'])['hours_of_stay']
    .mean()
    .round(1)
    .reset_index()
    .sort_values(by='hours_of_stay', ascending=False)
)

print("Average hours of ICU stay by ethnicity:")
print(stay_by_eth)

Average hours of ICU stay by ethnicity:
                                           ethnicity  hours_of_stay
6                                   UNABLE TO OBTAIN          320.6
0  AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...          272.1
2                             BLACK/AFRICAN AMERICAN          184.2
3                                 HISPANIC OR LATINO          179.0
7                              UNKNOWN/NOT SPECIFIED          118.2
8                                              WHITE           99.1
1                                              ASIAN           93.4
4                     HISPANIC/LATINO - PUERTO RICAN           77.8
5                                              OTHER           22.2


In [88]:
stay_by_gend = (
    ethnicity_gend_stay
    .dropna(subset=['hours_of_stay']) 
    .groupby(['gender'])['hours_of_stay']
    .mean()
    .round(1)
    .reset_index()
    .sort_values(by=['gender', 'hours_of_stay'], ascending=[False, False])
)

print("Average hours of ICU stay by gender:")
print(stay_by_gend)

Average hours of ICU stay by gender:
  gender  hours_of_stay
1      M           84.3
0      F          133.0
